In [62]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer, AutoTokenizer
from datasets import Dataset,DatasetDict
from transformers import default_data_collator
import pandas as pd
import json

In [53]:
qa_herbert_model = AutoModelForQuestionAnswering.from_pretrained("allegro/herbert-base-cased")
tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.sso.sso_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.sso.sso_relationship.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized

In [55]:
def to_squad_format(dataset):
    res = {
        "id": [],
        "question": [],
        "answers": [],
        "context": [],
        "title": []
    }
    for act in dataset["data"]:
        title = act["title"]
        for para in act["paragraphs"]:
            ctx = para["context"]
            for q in para["qas"]:
                q["context"] = ctx
                q["title"] = title
                answers = {"answer_start": [], "text": []}
                for an in q["answers"]:
                    answers["answer_start"].append(an["answer_start"])
                    answers["text"].append(an["text"])
                q["answers"] = answers
                for c in ["id", "question", "answers", "context", "title"]:
                    res[c].append(q[c])
    return Dataset.from_dict(res)
    
def read_dataset():
    dataset = {}
    for t in ["train", "dev", "test"]:
        with open(f"lqad-pl-1/lqad-pl-{t}.json") as f:
            dataset[t] = to_squad_format(json.loads(f.read()))
    return DatasetDict(dataset)

dataset = read_dataset()


In [56]:
dataset["train"][0]

{'id': '2007_adw_1',
 'question': 'Wybrany do Sejmu może być obywatel polski mający prawo wybierania, który: ',
 'answers': {'answer_start': [115],
  'text': ['najpóźniej w dniu wyborów kończy 21 lat']},
 'context': ' Rozdział IV SEJM I SENAT Art . 99 § 1 . Wybrany do Sejmu może być obywatel polski mający prawo wybierania , który najpóźniej w dniu wyborów kończy 21 lat . § 2 . Wybrany do Senatu może być obywatel polski mający prawo wybierania , który najpóźniej w dniu wyborów kończy 30 lat . § 3 . Wybraną do Sejmu lub do Senatu nie może być osoba skazana prawomocnym wyrokiem na karę pozbawienia wolności za przestępstwo umyślne ścigane z oskarżenia publicznego .',
 'title': 'Konstytucja Rzeczypospolitej Polskiej z dnia 2 kwietnia 1997 r. uchwalona przez Zgromadzenie Narodowe w dniu 2 kwietnia 1997 r., przyjęta przez Naród w referendum konstytucyjnym w dniu 25 maja 1997 r., podpisana przez Prezydenta Rzeczypospolitej Polskiej w dniu 16 lipca 1997 r'}

In [57]:
# src: https://huggingface.co/docs/transformers/custom_datasets#load-squad-dataset
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [58]:
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
data_collator = default_data_collator

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=qa_herbert_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 3719
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 699


Epoch,Training Loss,Validation Loss
